In [60]:
import json

In [61]:
with open('data/5_kings_all.json', 'r') as f:
    qa_dataset = json.loads(f.read())

with open('data/knowledge_graph_query_result.json', 'r') as f:
    knowledge_graph_query_result_set = json.loads(f.read())

In [62]:
# human_eval_set = []

# id = 0
# for qa, knowledge_graph_query_result, model_result in zip(qa_dataset, knowledge_graph_query_result_set, gpt4_result_set):
#     human_eval_set.append({
#         'id': id,
#         'context': qa['context'],
#         'question': qa['question'],
#         'answer': qa['answer'],
#         'model_result': model_result,
#     })

#     id += 1

# human_eval_set
# with open('data/human_set.json', 'w') as f:
# f.write(json.dumps(human_eval_set, indent=4))

In [63]:
def eval(model_result_set):
    metric = {
        'correct': {
            'in_knowledge_graph': 0,
            'not_in_knowledge_graph': 0,
        },
        'incorrect': {
            'in_knowledge_graph': 0,
            'not_in_knowledge_graph': 0,
        },
    }
    for qa, knowledge_graph_query_result, model_result in zip(qa_dataset, knowledge_graph_query_result_set, model_result_set):
        mask = metric

        found_in_answer = False
        answer = qa['answer']
        if any(token in answer for token in model_result.replace('.', ' ').strip().split()):
            found_in_answer = True
        mask = mask['correct'] if found_in_answer else mask['incorrect']

        found_in_knoledge_graph = False
        for entity in knowledge_graph_query_result:
            if any(token in entity for token in model_result.replace('.', ' ').strip().split()):
                found_in_knoledge_graph = True
                break
        if found_in_knoledge_graph:
            mask['in_knowledge_graph'] += 1
        else:
            mask['not_in_knowledge_graph'] += 1

    model_accuracy = (metric['correct']['in_knowledge_graph'] + metric['correct']['not_in_knowledge_graph']) / \
        (metric['correct']['in_knowledge_graph'] +
         metric['correct']['not_in_knowledge_graph'] +
         metric['incorrect']['in_knowledge_graph'] +
         metric['incorrect']['not_in_knowledge_graph'])

    print('# Model Result')
    print(f'Accurracy: {model_accuracy}')

    print('====================')

    tp = metric['correct']['in_knowledge_graph']
    fp = metric['incorrect']['in_knowledge_graph']
    fn = metric['correct']['not_in_knowledge_graph']
    tn = metric['incorrect']['not_in_knowledge_graph']

    print('# Knowledge Graph Result')
    print(f'TP: {tp}, FP: {fp}')
    print(f'FN: {fn}, TN: {tn}')
    print(f'Accuracy: {(tp + tn) / (tp + tn + fp + fn)}')
    if tp + fp == 0:
        print(f'Precision: NaN')
    else:
        print(f'Precision: {tp / (tp + fp)}')
    print(f'Recall: {tp / (tp + fn)}')
    print(f'F1: {2 * tp / (2 * tp + fp + fn)}')

In [87]:
with open('data/llama2_result.txt', 'r') as f:
    result = f.read().split('\n')
    eval(result)

# Model Result
Accurracy: 0.26693227091633465
# Knowledge Graph Result
TP: 31, FP: 14
FN: 36, TN: 170
Accuracy: 0.8007968127490039
Precision: 0.6888888888888889
Recall: 0.4626865671641791
F1: 0.5535714285714286


In [88]:
with open('data/llama2_fine_tuned_result.txt', 'r') as f:
    result = f.read().split('\n')
    eval(result)

# Model Result
Accurracy: 0.2589641434262948
# Knowledge Graph Result
TP: 34, FP: 11
FN: 31, TN: 175
Accuracy: 0.8326693227091634
Precision: 0.7555555555555555
Recall: 0.5230769230769231
F1: 0.6181818181818182


In [89]:
with open('data/gpt3_5_result.txt', 'r') as f:
    result = f.read().split('\n')
    eval(result)

# Model Result
Accurracy: 0.2788844621513944
# Knowledge Graph Result
TP: 34, FP: 15
FN: 36, TN: 166
Accuracy: 0.796812749003984
Precision: 0.6938775510204082
Recall: 0.4857142857142857
F1: 0.5714285714285714


In [90]:
with open('data/gpt3_5_fine_tuned_result.txt', 'r') as f:
    result = f.read().split('\n')
    eval(result)

# Model Result
Accurracy: 0.33067729083665337
# Knowledge Graph Result
TP: 36, FP: 9
FN: 47, TN: 159
Accuracy: 0.7768924302788844
Precision: 0.8
Recall: 0.43373493975903615
F1: 0.5625


In [91]:
with open('data/gpt4_result.txt', 'r') as f:
    result = f.read().split('\n')
    eval(result)

# Model Result
Accurracy: 0.3466135458167331
# Knowledge Graph Result
TP: 43, FP: 9
FN: 44, TN: 155
Accuracy: 0.7888446215139442
Precision: 0.8269230769230769
Recall: 0.4942528735632184
F1: 0.6187050359712231
